In [11]:
from pybase64 import urlsafe_b64decode, b64decode
from ipywidgets import Output
from io import BytesIO
from PIL import Image
import numpy as np
from ipycanvas import Canvas, hold_canvas

from associations_module import Associations
from sample_class_module import Sample


def index(request):
    ref_sample_b64 = request['ref_sample']
    test_samples_b64 = [s for s in request['test_samples']]

    ref_ndarray = get_image_ndarray(get_b64string(ref_sample_b64))
    test_ndarrays = [get_image_ndarray(get_b64string(s)) for s in test_samples_b64]

    ref_2darray = cvt_to_2darray(ref_ndarray, pick_last_value_in_list)
    test_2darrays = [cvt_to_2darray(s, pick_last_value_in_list) for s in test_ndarrays]
    return test_2darrays[compare_samples(ref_2darray, test_2darrays)]

def get_b64string(s):
    return s.split(",")[1]

def pick_last_value_in_list(a_list):
    return a_list[-1]

def get_image_ndarray(base64_image_string):
    image = Image.open(BytesIO(b64decode(base64_image_string)))
    return np.array(image.resize((10, 10)))

def cvt_to_2darray(numpy_ndarray, func):
    result = np.empty((len(numpy_ndarray), len(numpy_ndarray[0])))

    for row in range(len(numpy_ndarray)):
        for col in range(len(numpy_ndarray[0])):
            result[row, col] = func(numpy_ndarray[row, col])
    
    return result

def compare_samples(ref_2darray, test_2darrays):
    ref_association = Associations(Sample(ref_2darray, thinify=True).convertToAngle())
    test_1_association = Associations(sampleImage=Sample(test_2darrays[0]).convertToAngle(), usePlainDataframes=True, includeDistance=False, limitDistance=10)
    test_2_association = Associations(sampleImage=Sample(test_2darrays[1]).convertToAngle(), usePlainDataframes=True, includeDistance=False, limitDistance=10)

    comp1 = ref_association.compare(test_1_association)
    comp2 = ref_association.compare(test_2_association)

    # breakpoint()
 
    dist1 = comp1.getDistanceToNeighbors(reference='column')
    dist2 = comp2.getDistanceToNeighbors(reference='column')

    return int(dist1 < dist2)

In [2]:
canvas = Canvas(width=200, height=200)

In [27]:

out = Output()

painting = True

@out.capture()
def handle_mouse_move(x, y):
    print("what the fuck is going on")
    # if(painting):
    #     print("what the fuck")
    #     canvas.line_to(x, y)
    #     canvas.stroke('red')
    # else:
    #     print("what!")

@out.capture()
def handle_mouse_down(x, y):
    canvas.begin_path()
    canvas.move_to(x, y)

@out.capture()
def handle_mouse_up(x, y):
    canvas.close_path()

canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_up(handle_mouse_up)

In [29]:
display(canvas)

Canvas(width=500)

In [30]:
canvas

Canvas(width=500)